In [4]:
import pandas as pd
import numpy as np
import sys
import os
import re
import datetime 
import networkx
import json
from konlpy.tag import Komoran
tagger = Komoran()

In [17]:
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

In [6]:
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s


In [7]:
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

In [8]:
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

In [9]:
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))


In [10]:
def summarise_contents(x):
    tr = TextRank()

    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
    tr.loadSents(RawSentence(x), lambda sent: filter(lambda y:y not in stopword and y[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
    
    tr.build()
    ranks = tr.rank()
    return tr.summarize(0.2)

In [11]:
with open("data.json", 'r') as f:
    data = json.load(f)

In [20]:
content = data[1]["content"]

In [21]:
content

['회동이 끝난 후 야당 원내대표들은 김태우 특검법 도입과 신재민 전 사무관의 폭로를 검증하기 위한 기획재정위원회 소집·청문회 등을 더불어민주당에 요청했지만, 홍영표 민주당 원내대표(인천 부평을)가 수용 불가 입장을 밝혔다고 전했다.',
 '나경원 한국당 원내대표는 "오늘 가장 강하게 요청한 것은 특검법이었다"고 말했다.',
 '나 원내대표는 "김태우 수사관이 제기한 의혹은 운영위에서 차고 넘치는 증거를 발견할 수 있었다"라며 "이와 관련된 상임위 소집을 요구하고 있는데 실질적으로 여당이 전혀 협조하고 있지 않다"라고 지적했다.',
 '이어 "신재민 전 사무관이 제기한 의혹에 대해서도 기재위가 실질적으로 김동연 전 경제부총리를 부를 수 있어야 할 것"이라며 "지난번 운영위 진행에서 느낄 수 있었지만 일방적 상임위에 한계를 느꼈기 때문에 청문회를 진행하는 것에 대해서도 논의하고 싶다"고 덧붙였다.',
 '김관영 바른미래당 원내대표도 "신재민 전 사무관 건은 기재위 차원의 청문회를 일관되게 저희 당이 주장해왔다"라며 "전 국회가 국민적 의혹이 있는 사안에 대해 신속하게 청문회 내지는 위원회 열어서 그런 점들에 대해 의혹을 규명하는 게 필요하다고 생각한다"라고 밝혔다.',
 '이어 "국회에서 현안 관련 상임위 개최 청문회, 소규모 단위 국정조사는 자주 개최돼야 한다"고 강조했다.',
 '김 원내대표는 "국회선진화법 개정, 법사위 개선 등 문제들을 운영위 제도 개선 소위에서 신속히 논의해서 개선이 돼야 할 것"이라며 "선거법 제도 개혁, 법안소위 주 1회 소집, 특별감찰관 임명문제, 채용비리 국정조사 계획서 문제 등이 2월 국회에서 꼭 결실 맺을 수 있게 해야 할 것"이라고도 했다.',
 '반면 홍영표 민주당 원내대표는 말을 아꼈다.',
 '홍 원내대표는 모두발언을 통해 "새해를 맞아서 국회가 대화와 타협으로 민생을 챙기고 한반도 평화를 준비하는 국회가 될 수 있게 최선을 다하겠다"고 했다.',
 '한편 신재민 전 기획재정부 사무관을 원색적으로 비난한 손혜원 민주당 

In [19]:
summarise_contents(content)

이번 조사는 지난해 10월 15일~24일까지 서비스센터 업체를 이용한 성인 1268명을 대상으로 온라인설문 방식으로 진행됐다.
서비스 품질·상품·호감도 3개 부문을 총체적으로 평가한 종합만족도는 국산차 서비스센터가 평균 3.60점, 수입차 서비스센터가 평균 3.54점이었다.
세부 항목 평가 결과를 살펴보면 서비스센터 서비스의 일관성, 전문성, 고객 대응 등 '서비스 품질'에 대한 만족도는 국산차 서비스센터의 경우 평균 3.77점이었다.
수입차 서비스센터는 평균 3.72점으로, 렉서스 3.93점, 메르세데스 벤츠 3.86점, 포드 3.80점 등의 순이었다.
'서비스 상품 만족도'는 시설 이용 편리성, 점검절차 및 비용의 적절성, 사이트·앱 정보유용성 3개 요인으로 평가했는데, 국산·수입차 서비스센터 모두 시설 이용 편리성 만족도가 각각 3.71점과 3.88점으로 높은 반면, 사이트·앱 정보유용성 만족도는 3.29점으로 낮았다.
국산차 서비스센터의 경우 르노삼성이 모든 요인에서 만족도가 상대적으로 높았고, 수입차 서비스센터는 렉서스가 점검절차 및 비용의 적절성에서, 벤츠가 시설 이용 편리성에서 만족도가 상대적으로 높았다.
소비자가 서비스를 체험하면서 느낀 주관적 감정을 평가한 '서비스 호감도'는 국산차 서비스센터의 경우 르노삼성 3.69점, 쌍용자동차 3.62점 등의 순이었다.
한편 피해구제 합의율은 BMW와 벤츠가 상대적으로 높았다.
지난 2015년부터 최근 3년간 소비자원에 접수된 11개 자동차 서비스센터 관련 피해구제 신청건수는 총 916건으로 집계됐다.
등록대수 10만 대당 건수는 현대자동차가 1.4건으로 가장 적었고 다음으로 렉서스, 기아자동차 등의 순이었다.
피해구제 접수사건 중 환급, 배상, 계약해제 등 보상 처리된 사건의 비율(합의율)은 BMW와 벤츠가 각각 66.7%, 57.8%인데 비해 아우디(38.2%), 쌍용자동차(49.0%)의 합의율은 상대적으로 낮게 나타났다.
소비자원 관계자는 "이번 조사 결과를 관련 사업자와 공유하고 취약부문에 대한 개

'수입차 서비스센터는 평균 3.72점으로, 렉서스 3.93점, 메르세데스 벤츠 3.86점, 포드 3.80점 등의 순이었다. 국산차 서비스센터의 경우 르노삼성이 모든 요인에서 만족도가 상대적으로 높았고, 수입차 서비스센터는 렉서스가 점검절차 및 비용의 적절성에서, 벤츠가 시설 이용 편리성에서 만족도가 상대적으로 높았다.'

In [22]:
summarise_contents(content)

회동이 끝난 후 야당 원내대표들은 김태우 특검법 도입과 신재민 전 사무관의 폭로를 검증하기 위한 기획재정위원회 소집·청문회 등을 더불어민주당에 요청했지만, 홍영표 민주당 원내대표(인천 부평을)가 수용 불가 입장을 밝혔다고 전했다.
나경원 한국당 원내대표는 "오늘 가장 강하게 요청한 것은 특검법이었다"고 말했다.
나 원내대표는 "김태우 수사관이 제기한 의혹은 운영위에서 차고 넘치는 증거를 발견할 수 있었다"라며 "이와 관련된 상임위 소집을 요구하고 있는데 실질적으로 여당이 전혀 협조하고 있지 않다"라고 지적했다.
이어 "신재민 전 사무관이 제기한 의혹에 대해서도 기재위가 실질적으로 김동연 전 경제부총리를 부를 수 있어야 할 것"이라며 "지난번 운영위 진행에서 느낄 수 있었지만 일방적 상임위에 한계를 느꼈기 때문에 청문회를 진행하는 것에 대해서도 논의하고 싶다"고 덧붙였다.
김관영 바른미래당 원내대표도 "신재민 전 사무관 건은 기재위 차원의 청문회를 일관되게 저희 당이 주장해왔다"라며 "전 국회가 국민적 의혹이 있는 사안에 대해 신속하게 청문회 내지는 위원회 열어서 그런 점들에 대해 의혹을 규명하는 게 필요하다고 생각한다"라고 밝혔다.
이어 "국회에서 현안 관련 상임위 개최 청문회, 소규모 단위 국정조사는 자주 개최돼야 한다"고 강조했다.
김 원내대표는 "국회선진화법 개정, 법사위 개선 등 문제들을 운영위 제도 개선 소위에서 신속히 논의해서 개선이 돼야 할 것"이라며 "선거법 제도 개혁, 법안소위 주 1회 소집, 특별감찰관 임명문제, 채용비리 국정조사 계획서 문제 등이 2월 국회에서 꼭 결실 맺을 수 있게 해야 할 것"이라고도 했다.
반면 홍영표 민주당 원내대표는 말을 아꼈다.
홍 원내대표는 모두발언을 통해 "새해를 맞아서 국회가 대화와 타협으로 민생을 챙기고 한반도 평화를 준비하는 국회가 될 수 있게 최선을 다하겠다"고 했다.
한편 신재민 전 기획재정부 사무관을 원색적으로 비난한 손혜원 민주당 의원이 여론의 뭇매를 맞고 있다.


'나경원 한국당 원내대표는 "오늘 가장 강하게 요청한 것은 특검법이었다"고 말했다. 반면 홍영표 민주당 원내대표는 말을 아꼈다.'